In [51]:
import threading
import math
import matplotlib.pyplot as plt
import requests
from decimal import *
import pandas as pd
from binance.client import Client
global client
client = Client("V40EnBQtXZZLdxv7r7DmVqfJ6NFGdHn1Wp8OhTr9Nq5BnpQNl1qqdRZORGRsaYTu", "5v6pMJ3r0VmkFrA2WrghNjLLdNLkkX9mCULmWvafPdmlEEu7RByGoVxNzXDmp6")
# Don't forget to complete private key before api call
import pandas as pd
import numpy as np
import plotly
plotly.tools.set_credentials_file(username='Sethi', api_key='ZvmgzOHt0HBDU1N8JPLg')
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from datetime import datetime

In [73]:
def getCoinBalance(coinSymbol, free_or_locked = 'free'):
    info = client.get_account()
    balances = info['balances']
    for coin in balances:
        if coin['asset'] == coinSymbol:
            return coin[free_or_locked]

class Candle:
    def __init__(self, ts, open, high, low, close): # ts: timestamp
        self.ts = ts
        self.open = open
        self.high = high
        self.low = low
        self.close = close
    def printCandle(self):
        print('TimeStamp: '+str(self.ts)+' Close: '+str(self.close))
    '''
    def changeType(self, candleType):
        if candleType == "HA":
            print("Hacienda Candle Code")
    '''
class Chart:
    def __init__(self, candles = None, candleType = 'binance'):
        self.candles = []
        self.indicators = {}
        if candles != None:
            if candleType == 'binance':
                for candle in candles:
                    self.candles.append( Candle(candle[0],candle[1],candle[2],candle[3],candle[4]))
            elif candleType == 'candles':
                self.candles = candles
        self.series = {
            'xAxis': [c.ts for c in self.candles],
            'open': [c.open for c in self.candles],
            'high': [c.high for c in self.candles],
            'low': [c.low for c in self.candles],
            'close': [c.close for c in self.candles]
        }
        
    def printChart(self):
        for candle in self.candles:
            candle.printCandle()
    
    def addIndicator(self, name, values): # name; key for dict as string, values: array of indicator values for each candle 
        self.indicators[name] = values
    
    def ma(self, length, maType='sma', dataType='close'):
        """ Moving Average
        maType: type of MA ex. 'sma', 'ema'
        dataType: type of values taken ex. 'close', 'ohlc' (Open, high, low, and close)
        length: length of the MA
        Returns the value of the MA for the last Candle"""
        # klines = client.get_historical_klines("ETHUSDT", Client.KLINE_INTERVAL_1MINUTE, "1 hour ago UTC")
        candles = self.candles
        #for i in range(offset):
        #    candles.pop()
        #print(candles[-1].close)
        if dataType == 'close':
            points = pd.Series(data = self.series['close'])
            #print(list(points.ewm(span=length).mean())[-1])
        elif dataType == 'ohlc':
            length = int(length * 4)
            myArray = []
            for candle in candles:
                myArray.append(candle.open)
                myArray.append(candle.high)
                myArray.append(candle.low)
                myArray.append(candle.close)
            points = pd.Series(data = myArray)
            
        finalArray = []
        if maType == 'ema':
            finalArray = list(points.ewm(span=length).mean())
        elif maType == 'sma':
            finalArray = list(points.rolling(window=length).mean())
        for i in range(len(finalArray)):
            finalArray[i] = round(finalArray[i],3)
        maName = str(length)+' '+str(maType)
        if dataType != 'close':
            maName += ' ' + str(dataType)
        self.addIndicator( maName, finalArray)
        return finalArray 

    def addMacd(self):
        ma1 = self.ma(12,'ema') 
        ma2 = self.ma(26,'ema')
        macd = np.subtract(ma2,ma1) # ma2 - ma1 
        """for i in range(len(ma2)):
            macd.append(ma2[i]-ma1[i])"""
        self.addIndicator('macd', macd) 
        return macd
    def plot(self):
        #for iName, iVals in self.indicators.items():
        sma10 = go.Scatter(x = self.series['xAxis'], y = self.ma(10)[10:])
        trace = go.Candlestick(x=self.series['xAxis'],
                               open=self.series['open'],
                               high=self.series['high'],
                               low=self.series['low'],
                               close=self.series['close'])
        layout = {
            'title': 'AAPL Chart',
            'yaxis': {'title': 'Price'}
        }
        fig = dict(data=[ trace, sma10 ], layout=layout)
        return py.iplot(fig, filename='simple_candlestick')
        
class Symbol:
    def __init__(self, name, usdAmt):
        # string name: of Symbol ex."BTCUSDT", float usdAmt: dollars allocated to this symbol
        self.info = client.get_symbol_info(name)
        prices = client.get_all_tickers()
        
        self.name = name
        self.buys = []
        self.sells = []
        self.price = float([x['price'] for x in prices if x['symbol'] == self.name][0])
        self.lotSize = float(self.info['filters'][1]['stepSize'])
        self.charts = {}
        #       CHANGE HERE chart length
        self.charts['H1'] = Chart(
            client.get_historical_klines(self.name, Client.KLINE_INTERVAL_1HOUR, "100 hour ago UTC"))
        self.charts['D1'] = Chart(
            client.get_historical_klines(self.name, Client.KLINE_INTERVAL_1DAY, "100 day ago UTC"))
        self.charts['M1'] = Chart(
            client.get_historical_klines(self.name, Client.KLINE_INTERVAL_1MINUTE, "100 minute ago UTC"))
        self.charts['W1'] = Chart(
            client.get_historical_klines(self.name, Client.KLINE_INTERVAL_1WEEK, "100 week ago UTC"))
        
        if self.info["quoteAsset"] == 'USDT':
            self.amt = usdAmt
        else:
            self.amt = usdAmt * self.price 
                    # USD Amount * current price of quote asset (ex. BTC, ETH) in USD
        self.amt = self.amt - (float(self.amt) % float(self.info['filters'][1]['stepSize']))
            # Round amount down to nearest lotsize
            
        print("Lotsize: " + str(self.lotSize))
        print("Amount: " + str(self.amt) + " " + str(self.info["quoteAsset"]))
        print("Init Symbol " + name)
        
    def buy(self):
        self.buys.append(curPrice*1.0005)
        print("buy " + s(self.amt) + " at "+s(buys[-1]))
        order = client.order_market_buy(
            symbol= self.name,
            quantity=self.amt)
    def getInfo(self):
        self.info = client.get_symbol_info(self.name)
        return self.info
    
    def getPrice(self):
        prices = client.get_all_tickers()
        self.price = float([x['price'] for x in prices if x['symbol'] == self.name][0])
        return self.price
    
    def ma(self, length, maType, dataType, candleTF, offset):
        """ Moving Average
        maType: type of MA ex. 'sma', 'ema'
        dataType: type of values taken ex. 'close', 'ohlc' (Open, high, low, and close)
        length: length of the MA
        candleTF: TimeFrame of the candles ex. 'H1', 'D1' (1 Day candles)
        Returns the value of the MA for the last Candle"""
        # klines = client.get_historical_klines("ETHUSDT", Client.KLINE_INTERVAL_1MINUTE, "1 hour ago UTC")
        candles = self.charts[candleTF].candles
        for i in range(offset):
            candles.pop()
        print(candles[-1].close)
        if dataType == 'close':
            points = pd.Series(data = [candle.close for candle in candles])
            print(list(points.ewm(span=length).mean())[-1])
        elif dataType == 'ohlc':
            length = int(length * 4)
            myArray = []
            for candle in candles:
                myArray.append(candle.open)
                myArray.append(candle.high)
                myArray.append(candle.low)
                myArray.append(candle.close)
            points = pd.Series(data = myArray)
        finalValue = 0.0
        if maType == 'ema':
            finalValue = list(points.ewm(span=length).mean())[-1]
        elif maType == 'sma':
            finalValue = list(points.rolling(window=length).mean())[-1]
        return round(float(finalValue),4)




In [134]:
response = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AAPL&outputsize=full&interval=1day&apikey=W53NMAU4M50I88ED')
x = response.json()
myCandles = []
for ts, ci in x['Time Series (Daily)'].items(): # ts: TimeStamp/Date String , ci: Candle Info
    myCandles.append(Candle(ts, ci['1. open'], ci['2. high'], ci['3. low'], ci['4. close']))
AAPL_Chart = Chart(myCandles, 'candles')

AAPL_Chart.ma(10)
AAPL_Chart.addMacd()
AAPL_Chart.plot()

In [135]:
self = AAPL_Chart
sma10 = go.Scatter(x = self.series['xAxis'], y = self.ma(10)[10:])
trace = go.Candlestick(x=self.series['xAxis'],
                       open=self.series['open'],
                       high=self.series['high'],
                       low=self.series['low'],
                       close=self.series['close'])
layout = {
    'title': 'AAPL Chart',
    'yaxis': {'title': 'Price'}
}
fig = dict(data=[ trace, sma10 ], layout=layout)
py.iplot(fig, filename='simple_candlestick')

In [148]:
# Fitting Keras Model on AAPL Data

candles = AAPL_Chart.candles
macd = AAPL_Chart.indicators['macd']
sma10 = AAPL_Chart.indicators['10 sma']
print(macd)
inputData = []
results = [] # one hot format array of tuples for neural network output
             # (1,0,0): price will go down, sell; (0,1,0): insignificant change, wait;  (0,0,1): will go up, buy
for i, candle in enumerate(candles[1:]):       # candles[i] is the previous candle whereas candle is current
    close = float(candle.close)
    lastClose = float(candles[i].close)
    low = float(candle.low)
    v = macd[i-1] / macd[i]
    w = close / sma10[i-1] -1
    x = math.log(close) - math.log(lastClose)
    y = (close - lastClose) / lastClose
    z = (close - low) / (float(candle.high) - low)   # normalized data as 3 input variables
    inputData.append([v, w, x, y, z])
    if y > 0.004:
        result = (0,0,1)
    elif y < -0.004:
        result = (1,0,0)
    else:
        result = (0,1,0)
    results.append(result)
    
inputData = np.array(inputData[26:])   # macd n/a for first 26 values
results = np.array(results[26:])
print(results.shape)
print(inputData.shape)
print(inputData[-10:])
#print(inputData)
 
print('shape: ' + str(results.shape))

# split data into test and training
train_data = inputData[:5000]
test_data = inputData[5000:]
train_predictions= results[:5000]
test_predictions= results[5000:]

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
'''
model = Sequential([
    Dense(32, input_shape=(4,)),
    Activation('relu'),
    Dense(1),
    Activation('softmax'),
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
'''
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(5,)))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(120, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#'''
model.fit(train_data, train_predictions, epochs=500, batch_size=5000, validation_data=(test_data, test_predictions))

[ 0.     0.062  0.143 ... -0.316 -0.026  0.281]
(5955, 3)
(5955, 5)
[[ 8.88888889e-01  6.23313997e-02 -1.11733006e-02 -1.11111111e-02
   2.50000000e-01]
 [ 9.62313760e-01  5.97152369e-02  8.50307204e-03  8.53932584e-03
   2.85714286e-01]
 [ 9.56412406e-01  6.05777321e-02  1.10792181e-02  1.11408200e-02
   3.86503067e-01]
 [ 9.47577442e-01  7.99759749e-02  2.97427764e-02  3.01895108e-02
   7.56052142e-01]
 [ 8.95448080e-01 -4.23921962e-03 -6.76950128e-02 -6.54545455e-02
   8.87272727e-01]
 [ 1.11410460e+00 -7.31784132e-02 -5.86809866e-02 -5.69924468e-02
   2.27272727e-01]
 [ 1.34973262e+00 -5.95400703e-02  1.92313619e-02  1.94174757e-02
   4.35000000e-01]
 [ 1.27732240e+00 -1.27351080e-01 -7.71896387e-02 -7.42857143e-02
   2.06349206e-01]
 [ 2.31645570e+00 -1.14877166e-01  1.27780936e-02  1.28600823e-02
   7.50000000e-01]
 [ 1.21538462e+01 -1.23564203e-01 -2.57215826e-02 -2.53936008e-02
   5.00000000e-01]]
shape: (5955, 3)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning:

divide by zero encountered in double_scalars



Train on 5000 samples, validate on 955 samples
Epoch 1/500
5000/5000 [==============================] - 2s 488us/step - loss: 1.0942 - acc: 0.4136 - val_loss: 1.0354 - val_acc: 0.4251
Epoch 2/500
5000/5000 [==============================] - 0s 64us/step - loss: 1.0952 - acc: 0.4222 - val_loss: 1.0878 - val_acc: 0.4565
Epoch 3/500
5000/5000 [==============================] - 0s 64us/step - loss: 1.1034 - acc: 0.3810 - val_loss: 1.0754 - val_acc: 0.4314
Epoch 4/500
5000/5000 [==============================] - 0s 64us/step - loss: 1.0799 - acc: 0.4046 - val_loss: 1.0476 - val_acc: 0.4251
Epoch 5/500
5000/5000 [==============================] - 0s 68us/step - loss: 1.0673 - acc: 0.4210 - val_loss: 1.0271 - val_acc: 0.4251
Epoch 6/500
5000/5000 [==============================] - 0s 65us/step - loss: 1.0601 - acc: 0.4182 - val_loss: 1.0355 - val_acc: 0.4251
Epoch 7/500
5000/5000 [==============================] - 0s 71us/step - loss: 1.0638 - acc: 0.4240 - val_loss: 1.0216 - val_acc: 0.4387


Epoch 61/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.9513 - acc: 0.5776 - val_loss: 0.7763 - val_acc: 0.7026
Epoch 62/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.9466 - acc: 0.6060 - val_loss: 0.8623 - val_acc: 0.5822
Epoch 63/500
5000/5000 [==============================] - 0s 63us/step - loss: 0.9648 - acc: 0.5296 - val_loss: 0.7049 - val_acc: 0.7414
Epoch 64/500
5000/5000 [==============================] - 0s 65us/step - loss: 0.8145 - acc: 0.6502 - val_loss: 0.5902 - val_acc: 0.8094
Epoch 65/500
5000/5000 [==============================] - 0s 66us/step - loss: 0.6931 - acc: 0.7492 - val_loss: 0.5425 - val_acc: 0.8094
Epoch 66/500
5000/5000 [==============================] - 0s 66us/step - loss: 0.6503 - acc: 0.7486 - val_loss: 0.4966 - val_acc: 0.8199
Epoch 67/500
5000/5000 [==============================] - 0s 67us/step - loss: 0.6194 - acc: 0.7498 - val_loss: 0.5062 - val_acc: 0.7969
Epoch 68/500
5000/5000 [=================

Epoch 121/500
5000/5000 [==============================] - 0s 66us/step - loss: 0.4987 - acc: 0.7492 - val_loss: 0.3489 - val_acc: 0.8356
Epoch 122/500
5000/5000 [==============================] - 0s 65us/step - loss: 0.5074 - acc: 0.7612 - val_loss: 0.5329 - val_acc: 0.7424
Epoch 123/500
5000/5000 [==============================] - 0s 65us/step - loss: 0.7090 - acc: 0.6536 - val_loss: 0.3360 - val_acc: 0.8555
Epoch 124/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.4782 - acc: 0.7856 - val_loss: 0.3519 - val_acc: 0.8356
Epoch 125/500
5000/5000 [==============================] - 0s 63us/step - loss: 0.4788 - acc: 0.7668 - val_loss: 0.2960 - val_acc: 0.8639
Epoch 126/500
5000/5000 [==============================] - 0s 63us/step - loss: 0.4354 - acc: 0.8128 - val_loss: 0.3837 - val_acc: 0.7927
Epoch 127/500
5000/5000 [==============================] - 0s 67us/step - loss: 0.5326 - acc: 0.7202 - val_loss: 0.3672 - val_acc: 0.8262
Epoch 128/500
5000/5000 [=========

...

Epoch 241/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.2745 - acc: 0.8760 - val_loss: 0.1628 - val_acc: 0.9309
Epoch 242/500
5000/5000 [==============================] - 0s 66us/step - loss: 0.2472 - acc: 0.8990 - val_loss: 0.1975 - val_acc: 0.9047
Epoch 243/500
5000/5000 [==============================] - 0s 65us/step - loss: 0.2779 - acc: 0.8664 - val_loss: 0.2189 - val_acc: 0.9005
Epoch 244/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.3363 - acc: 0.8418 - val_loss: 0.3181 - val_acc: 0.8293
Epoch 245/500
5000/5000 [==============================] - 0s 65us/step - loss: 0.4432 - acc: 0.7780 - val_loss: 0.2432 - val_acc: 0.8880
Epoch 246/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.3658 - acc: 0.8224 - val_loss: 0.2217 - val_acc: 0.8901
Epoch 247/500
5000/5000 [==============================] - 0s 63us/step - loss: 0.3008 - acc: 0.8538 - val_loss: 0.1552 - val_acc: 0.9382
Epoch 248/500
5000/5000 [=========

Epoch 301/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.1663 - acc: 0.9504 - val_loss: 0.1008 - val_acc: 0.9644
Epoch 302/500
5000/5000 [==============================] - 0s 63us/step - loss: 0.1624 - acc: 0.9490 - val_loss: 0.1424 - val_acc: 0.9340
Epoch 303/500
5000/5000 [==============================] - 0s 66us/step - loss: 0.1930 - acc: 0.9156 - val_loss: 0.2164 - val_acc: 0.9152
Epoch 304/500
5000/5000 [==============================] - 0s 65us/step - loss: 0.3452 - acc: 0.8572 - val_loss: 0.4860 - val_acc: 0.7948
Epoch 305/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.6881 - acc: 0.7132 - val_loss: 0.1976 - val_acc: 0.9068
Epoch 306/500
5000/5000 [==============================] - 0s 63us/step - loss: 0.2875 - acc: 0.8744 - val_loss: 0.1581 - val_acc: 0.9382
Epoch 307/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.2155 - acc: 0.9216 - val_loss: 0.1159 - val_acc: 0.9696
Epoch 308/500
5000/5000 [=========

Epoch 361/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.1762 - acc: 0.9188 - val_loss: 0.2014 - val_acc: 0.9225
Epoch 362/500
5000/5000 [==============================] - 0s 65us/step - loss: 0.3206 - acc: 0.8726 - val_loss: 0.4063 - val_acc: 0.8157
Epoch 363/500
5000/5000 [==============================] - 0s 63us/step - loss: 0.5684 - acc: 0.7610 - val_loss: 0.1362 - val_acc: 0.9393
Epoch 364/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.2013 - acc: 0.9188 - val_loss: 0.1285 - val_acc: 0.9403
Epoch 365/500
5000/5000 [==============================] - 0s 65us/step - loss: 0.1715 - acc: 0.9340 - val_loss: 0.0971 - val_acc: 0.9654
Epoch 366/500
5000/5000 [==============================] - 0s 65us/step - loss: 0.1425 - acc: 0.9544 - val_loss: 0.1078 - val_acc: 0.9518
Epoch 367/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.1455 - acc: 0.9440 - val_loss: 0.1075 - val_acc: 0.9455
Epoch 368/500
5000/5000 [=========

Epoch 421/500
5000/5000 [==============================] - 0s 66us/step - loss: 0.4127 - acc: 0.8394 - val_loss: 0.4613 - val_acc: 0.8534
Epoch 422/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.7217 - acc: 0.7960 - val_loss: 0.3057 - val_acc: 0.8555
Epoch 423/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.4185 - acc: 0.8004 - val_loss: 0.1144 - val_acc: 0.9665
Epoch 424/500
5000/5000 [==============================] - 0s 63us/step - loss: 0.1551 - acc: 0.9514 - val_loss: 0.0941 - val_acc: 0.9749
Epoch 425/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.1296 - acc: 0.9698 - val_loss: 0.0828 - val_acc: 0.9812
Epoch 426/500
5000/5000 [==============================] - 0s 66us/step - loss: 0.1183 - acc: 0.9706 - val_loss: 0.0742 - val_acc: 0.9843
Epoch 427/500
5000/5000 [==============================] - 0s 66us/step - loss: 0.1044 - acc: 0.9738 - val_loss: 0.0676 - val_acc: 0.9822
Epoch 428/500
5000/5000 [=========

Epoch 481/500
5000/5000 [==============================] - 0s 63us/step - loss: 0.1295 - acc: 0.9660 - val_loss: 0.0821 - val_acc: 0.9791
Epoch 482/500
5000/5000 [==============================] - 0s 66us/step - loss: 0.1095 - acc: 0.9736 - val_loss: 0.0713 - val_acc: 0.9853
Epoch 483/500
5000/5000 [==============================] - 0s 65us/step - loss: 0.1009 - acc: 0.9734 - val_loss: 0.0642 - val_acc: 0.9822
Epoch 484/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.0905 - acc: 0.9766 - val_loss: 0.0585 - val_acc: 0.9843
Epoch 485/500
5000/5000 [==============================] - 0s 64us/step - loss: 0.0839 - acc: 0.9728 - val_loss: 0.0533 - val_acc: 0.9843
Epoch 486/500
5000/5000 [==============================] - 0s 68us/step - loss: 0.0768 - acc: 0.9758 - val_loss: 0.0490 - val_acc: 0.9843
Epoch 487/500
5000/5000 [==============================] - 0s 68us/step - loss: 0.0706 - acc: 0.9792 - val_loss: 0.0452 - val_acc: 0.9853
Epoch 488/500
5000/5000 [=========

In [26]:
candles = client.get_historical_klines('ETHUSDT', Client.KLINE_INTERVAL_1HOUR, "50 hour ago UTC")
#print(candles)
myChart = Chart(candles)
                 
print('10 SMA: ' + str(myChart.ma(10,'sma', 'ohlc')[-1]))

print('5 EMA: ' + str(myChart.ma(5,'ema', 'close')[-1]))

print('MACD: ' + str(myChart.addMacd()))


NameError: name 'addBinanceCandles' is not defined

In [58]:


# -------- SETTINGS --------------------------------------------------

symbol = "ETHBTC"
stopLoss = 0.035  # [0-1]

# api calls
prices = client.get_all_tickers()
candles = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, "1 month ago UTC")


# inits
buys = []
buyDates = []
sells = []
sellDates = []
listingNum = 0
sma_10 = []
sma_20 = []
sma_30 = []
ema = []
ema30 = []
ema_10 = 0.0
avg_10 = 0.0
avg_20 = 0.0
avg_30 = 0.0
last_ema = -1.0
last_ema30 = -1.0
uptrend = 0
last_high = 0
open_now = 0
market = 100 * float(candles[-1][1]) / float(candles[20][1])
print("\nopen: " + str(float(candles[20][1])) + " cur: " + str(float(candles[-1][1])))
print(str(datetime.fromtimestamp(candles[20][0]/1000)) + " to " + str(datetime.fromtimestamp(candles[-1][0]/1000)) +
     " market: " + str(round(market)) + "%")
profit = 0
kline = [[],[],[],[],[]]
for i in range(0, len(candles)-1):
    kline[0].append(datetime.fromtimestamp(candles[i][0]/1000)) # 0 is Timestamp
    if float(candles[i][4]) > last_high:             # 4 is close
        last_high = float(candles[i][4]);
    if i >= 20:
        avg_10 = 0.0
        for j in range(10):
            avg_10 += float(candles[i-j][4])
        avg_10 /= 10
        if last_ema == -1.0:
            last_ema = avg_10
        ema_10 = (float(candles[i][4]) - last_ema) * 2 / 11 + last_ema
        ema.append(ema_10)
        sma_10.append(avg_10)
        last_ema = ema_10
    if i >= 30:
        avg_20 = 0.0
        for k in range(20):
            avg_20 += float(candles[i-k][4])
        avg_20 /= 20
        sma_20.append(avg_20)
        avg_30 = 0.0
        for k in range(30):
            avg_30 += float(candles[i-k][4])
        avg_30 /= 30
        sma_30.append(avg_30)
        
        if last_ema30 == -1.0:
            last_ema30 = avg_30
        ema_30 = (float(candles[i][4]) - last_ema30) * 2 / 301 + last_ema30
        ema30.append(ema_30)
        last_ema30 = ema_30
        
        temp = uptrend
        if ema_10 > avg_20:
            uptrend = 1
        else:
            uptrend = 0
        open_now = len(buys) - len(sells)
        if open_now == 0 and temp > uptrend:
            #print("Buy " + str(datetime.fromtimestamp(candles[i][0]/1000)) + " at " + candles[i][4])
            last_high = 0;
            buys.append(float(candles[i][4]))
            buyDates.append(datetime.fromtimestamp(candles[i][0]/1000))
        if open_now == 1 and (float(candles[i][4]) < last_high *(1-stopLoss)): # Change here
            #print("Sell " + str(datetime.fromtimestamp(candles[i][0]/1000)) + " at " + candles[i][4])
            sells.append(round(last_high *0.99,5))
            sellDates.append(datetime.fromtimestamp(candles[i][0]/1000))
    for j in range(1,5):
        kline[j].append(candles[i][j])

roi = 1.0
print()
if open_now:
    sells.append(float(candles[i][4]))
print(buys)
print(sells)
print(range(len(sells)))
for i in range(0, len(sells)-1):
    roi *= sells[i] / buys[i]
    #print("roi " + str(i) + " " + str(roi))
print(symbol + "  Return: " + str(round(roi * 100)) + "%  Market: " + str(round(market)) + " Trades: " + str(len(buys)))

ma_10_x = kline[0][10:]
ma_20_x = kline[0][20:]
sma10 = go.Scatter(x = ma_10_x, y = sma_10)
sma20 = go.Scatter(x = ma_20_x, y = sma_20)
ema10 = go.Scatter(x = ma_10_x, y = ema)
trace = go.Candlestick(x=kline[0],
                       open=kline[1],
                       high=kline[2],
                       low=kline[3],
                       close=kline[4])
data = [trace, sma10, sma20, ema10]

def getShapes():
    shapes = []
    for x in buyDates:
        shapes.append({
        'x0': x, 'x1': x,
        'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
        'line': {'color': 'rgb(0,30,0)', 'width': 3}
    })
    for x in sellDates:
        shapes.append({
        'x0': x, 'x1': x,
        'y0': 0, 'y1': 0.8, 'xref': 'x', 'yref': 'paper',
        'line': {'color': 'rgb(30,0,0)', 'width': 3}
    })
    return shapes

layout = {
    #'title': 'The Great Recession',
    #'yaxis': {'title': 'AAPL Stock'},
    'shapes': getShapes()
}
fig = dict(data=data, layout=layout)

trades_returns = []
for x in range(0, len(sells)-1):
    trades_returns.append(round(sells[x] / buys[x], 4))
roi = 1.0
for x in trades_returns:
    roi *= x - 0.0005 
print(trades_returns)
print(roi)
py.iplot(fig, filename='simple_candlestick')




open: 0.038775 cur: 0.034134
2018-09-04 17:00:00 to 2018-10-03 20:00:00 market: 88%

[0.034567, 0.033646, 0.031565, 0.030596, 0.03227, 0.033147, 0.032903, 0.035547, 0.032886, 0.035195]
[0.03529, 0.03267, 0.0313, 0.03039, 0.03317, 0.03398, 0.0364, 0.03614, 0.0341, 0.03538]
range(0, 10)
ETHBTC  Return: 120%  Market: 88 Trades: 10
[1.0209, 0.971, 0.9916, 0.9933, 1.0279, 1.0251, 1.1063, 1.0167, 1.0369]
1.1945994974488108
